# Test Environment

In [64]:
# Modul um Würfel zu simulieren
import random
random.seed(1321345589)

# Modul zum vereinfachten Auslesen und Bearbeiten von Datentabellen
import pandas as pd
from pathlib import Path

# Um User Input zu verarbeiten
import regex as re

## Aux Funktionen zur Würfelsimulation

In [65]:
def w20(anzahl: int = 1) -> list[int]:
    """
    Simulation eines W20 Würfels. 

    : anzahl : Integer, Anzahl der Würfe
    : return : Liste mit Ergebnissen (Integer 1-20)
    """

    assert anzahl > 0

    wuerfe = [random.randint(1, 20) for _ in range(anzahl)]

    kritischer_erfolg(wuerfe)

    if len(wuerfe) == 1:
        return wuerfe[0]

    return wuerfe


def w6(anzahl: int = 1) -> list[int]:
    """
    Simulation eines W6 Würfels. 

    :anzahl: Integer, Anzahl der Würfe
    : return : Liste mit Ergebnissen (Integer 1-6)
    """

    assert anzahl > 0

    wuerfe = [random.randint(1, 6) for _ in range(anzahl)]

    kritischer_erfolg(wuerfe)

    if len(wuerfe) == 1:
        return wuerfe[0]

    return sum(wuerfe)


def kritischer_erfolg(wuerfe: list[int]) -> None:
    """
    Prüft bei Wurfproben ob ein kritischer (Miss-)Erfolg erzielt wurde.

    : wuerfe : Ergebnisliste, enthält Integer
    : return : None
    """

    if len(wuerfe) == 1:
        if 1 in wuerfe:
            print("Kritischer Erfolg!")
        elif 20 in wuerfe:
            print("Kritischer Misserfolg!")
    
    elif len(wuerfe) > 1:
        anzahl_einsen = [x for x in wuerfe if x == 1]
        anzahl_zwanziger = [x for x in wuerfe if x == 20]

        if len(anzahl_einsen) >= 2:
            print("Kritischer Erfolg der Fertigkeitsprobe!")

        elif len(anzahl_zwanziger) >= 2:
            print("Kritischer Misserfolg der Fertigkeitsprobe!")
    
    else:
        return None

## Aux Funktionen zur Datenverwaltung

In [66]:
def datei_zu_dataframe(path: str, index_column: int = 0) -> pd.DataFrame:
    """
    Liest eine excel Datei ein

    : path : String, Pfad zur Datei
    : index_column : Integer
    : return : pandas Dataframe
    """

    dataframe = pd.read_excel(path, index_col=index_column)
    dataframe.dropna(inplace=True)

    if "Eigenschaftsprobe" in dataframe.columns:
        dataframe["Eigenschaftsprobe"] = dataframe.Eigenschaftsprobe.apply(lambda x: x.split("/"))

    return dataframe


def datei_zu_dict(path: str) -> dict:
    """
    Liest eine excel Tabelle ein und gibt ein dictionary aus, wobei die Spaltenname der Tabelle
    die keys für das dictionary darstellen werden. 
    
    : path : String, Pfad zur Datei
    : return : dict
    """

    frame = datei_zu_dataframe(path)
    framedict = frame.T.to_dict(orient="records")[0]

    return framedict


def zeile_zu_dict(frame: pd.DataFrame, zeile: str, art: str):
    """
    Bildet eine einzelne Teile eines Dataframes als dictionary ab

    : frame : pd.Dataframe
    : zeile : String, muss Indexname einer Zeile sein
    : return : dict
    """

    assert zeile in frame.T.columns

    framedict = frame.T[zeile].to_dict()
    framedict[art] = zeile

    return framedict

In [67]:
# Einlesen der Datentabelle zu den Fertigkeiten
dsa5_talente = datei_zu_dataframe("data/dsa5_talente.xlsx", index_column=1)
dsa5_kampftechnik = datei_zu_dataframe("data/kampftechnik.xlsx")

dsa5_waffen = datei_zu_dataframe("data/waffen.xlsx")
dsa5_ruestungen = datei_zu_dataframe("data/ruestungen.xlsx")
dsa5_zauber = datei_zu_dataframe("data/zauber.xlsx")

# Glossar einlesen und mit dem gespiegelten selbst erweitern
glossar = datei_zu_dict("data/glossar.xlsx")
glossar = glossar | {val: key for key, val in glossar.items()}

# Einlesen von Helden
margot_data = datei_zu_dict("data/held_margot.xlsx")
margot_inv = datei_zu_dataframe("data/held_margot_inventar.xlsx")
elsbeth_data = datei_zu_dict("data/held_elsbeth.xlsx")
elsbeth_inv = datei_zu_dataframe("data/held_elsbeth_inventar.xlsx")

In [68]:
dsa5_kampftechnik

,Leiteigenschaft,Steigerungsfaktor,Wert,AT,PA
Kampftechnik,,,,,
Armbrüste,FF,B,6,6,6
Bögen,FF,C,6,6,6
Dolche,GE,B,6,7,6
Fechtwaffen,GE,C,6,7,6
Hiebwaffen,KK,C,6,6,6
Kettenwaffen,KK,C,6,6,6
Lanzen,KK,B,6,6,6
Raufen,GE/KK,B,6,7,6
Schilde,KK,C,6,6,6


## Einheit

In [69]:
class Einheit():
    def __init__(self, heldenname: str):
        if f"data/held_{heldenname}.xlsx" is not None:
            file = datei_zu_dict(f"data/held_{heldenname}.xlsx")

            for key, value in file.items():
                setattr(self, key, value)
        
        self.inventar = datei_zu_dataframe(f"data/held_{heldenname}_inventar.xlsx")
        
        self.waffe_ausruesten()
        self.ruestung_tragen()
        self.zauber_einstimmen()
    

    def waffe_ausruesten(self, waffe: str = None) -> None:
        """
        Der Held soll in unserem Falle nicht ohne aktive Waffe starten.
        Falls die Haupthand 'Zauber' ist, können Zauber gewirkt werden.

        : waffe : String, Name der Waffe welche ausgerüstet werden soll
        : return : None
        """

        # Beim Initialisieren des Charakters wird diese Funktion ohne zusätzlichen
        # Parameter (also waffe = None) aufgerufen. Daher wird die erste 
        if waffe == None:
            waffe = self.inventar[self.inventar["Art"] == "Waffe"].iloc[0].name

        assert waffe in dsa5_waffen.T.columns.to_list()

        framedict = zeile_zu_dict(dsa5_waffen, waffe, "Waffe")
        self.haupthand = framedict


    def ruestung_tragen(self, ruestung: str = None):
        """
        
        """

        if ruestung == None:
            ruestung = self.inventar[self.inventar["Art"] == "Rüstung"].iloc[0].name

        assert ruestung in dsa5_ruestungen.T.columns.to_list()

        framedict = zeile_zu_dict(dsa5_ruestungen, ruestung, "Ruestung")
        self.ruestung = framedict
    

    def zauber_einstimmen(self, zauber: str = None):
        pass


    def _schaden_nehmen(self, schaden: int):
        assert schaden > 0

        self.lp -= schaden
        
        if self.lp <= 0:
            print(f"{self.name} ist kampfunfähig!")
            self.lp = 0
     
    
    def _angriffscheck(self) -> tuple:
        """
        
        """

        versuch = w20()
        probe = getattr(self, self.haupthand["L+S"])

        erfolgswert = probe + self.haupthand["AT-Mod"] - versuch
        erfolg = [True if erfolgswert > 0 else False][0]

        return (erfolg, erfolgswert, versuch)


    def _paradecheck(self) -> tuple:
        """
        
        """

        versuch = w20()
        probe = getattr(self, self.haupthand["L+S"])

        erfolgswert = probe + self.haupthand["PA-Mod"] - versuch
        erfolg = [True if erfolgswert > 0 else False][0]

        return (erfolg, erfolgswert, versuch)
    

    def schadensverteilung(self, other):
        schaden = w6(self.haupthand["TP"])
        ruestungsklasse = other.ruestung["Rüstungsschutz"]

        gemachter_schaden = [schaden - ruestungsklasse if schaden > ruestungsklasse else 0][0]

        return gemachter_schaden


    def kampfhandlung(self, other):
        at_erfolg, at_wert, at_wurf = self._angriffscheck()
        pa_erfolg, pa_wert, pa_wurf = other._paradecheck()

        if at_wurf == 1:
            schaden = self.schadensverteilung(other)

            print(f"Kritischer Erfolg der Angriffsprobe. {self.name} trifft {other.name} für {schaden} Schadenspunkte.")
        
        elif pa_wurf == 1:
            schaden = other.schadensverteilung(self)

            print(f"Kritischer Erfolg der Parade. {other.name} trifft {self.name} für {schaden} Schadenspunkte.")
        
        elif at_erfolg:
            if pa_erfolg:
                print(f"{other.name} pariert den erfolgreichen Angriff von {self.name}.")

                return
            
            else:
                schaden = self.schadensverteilung(other)
                print(f"{self.name} trifft {other.name} für {schaden} Schadenspunkte.")
        
        else:
            print(f"Angriff trifft {other.name} nicht. Geh in die Ecke und schäm dich.")


    def eigenschaftsprobe(self, probe: str, bias: int=0):
        wurf = w20()

        if wurf > getattr(self, glossar[probe]) + bias:
            print(f"Eigenschaftsprobe misslungen! Der nötige {probe}-Wert von {getattr(self, glossar[probe])} wurde durch {wurf} nicht unterworfen!")
        
        else:
            print(f"Eigenschaftsprobe geschafft! Der nötige {probe}-Wert von {getattr(self, glossar[probe])} wurde durch {wurf} unterworfen!")
    

    def fertigkeitsprobe(self, probe: str, bias: int=0):
        """
        
        """

        ausgleich = random.randint(3, 7)
        print(f'Starte Fertigkeitsprobe {probe} auf {dsa5_talente["Eigenschaftsprobe"][probe]}, Ausgleich: {ausgleich}')

        for i, eigenschaft in enumerate(dsa5_talente["Eigenschaftsprobe"][probe]):
            wurf = w20()

            ziel = getattr(self, eigenschaft) + bias
            wurfdifferenz = ziel - wurf

            if wurfdifferenz < 0:
                ausgleich += wurfdifferenz
            
                if ausgleich < 0:
                    print(f'\nFertigkeitsprobe misslungen!')

                    return
        
        print(f'\nFertigkeitsprobe geglückt! Übriger Ausgleich bei {ausgleich}.')

In [70]:
margot = Einheit("margot")
elsbeth = Einheit("elsbeth")
josran = Einheit("josran")

In [71]:
margot.kampfhandlung(elsbeth)

Angriff trifft Elsbeth nicht. Geh in die Ecke und schäm dich.
